### In this file, we consider the PDE system $\mathcal{L} \mathbf{u} = \mathbf{f}$ with zero Dirichlet boundary condition, where
$$
\mathcal{L}=\left[\begin{array}{cc}
1 & -\lambda \Delta \\
\lambda \Delta & 1
\end{array}\right],
\quad
\mathbf{u}=\left[\begin{array}{c} u_1 \\ u_2 \end{array}\right]
\quad
\mathbf{f}=\left[\begin{array}{c} f_1 \\ f_2 \end{array}\right]
$$

In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    gpu_info = torch.cuda.get_device_properties(0)
    print(f"GPU: {gpu_info.name}")
    print(f"GPU memory: {gpu_info.total_memory / 1024**2:.2f} MB")

GPU: NVIDIA GeForce RTX 3080 Ti
GPU memory: 12038.19 MB


In [3]:
n = 40
h = 1 / n 
m = n - 1
m2 = m * m
lambda_ = 0.05

In [4]:
id_m = np.identity(m)
d1= np.identity(m)
d1[0][1] = 1
d1[m - 1][m - 2] = 1
for i in range(0, m):
    d1[i][i] = -2
    if (i >= 1) and (i <= m - 2):
        d1[i][i - 1] = 1
        d1[i][i + 1] = 1
D = np.kron(id_m, d1) + np.kron(d1, id_m)
D = D / (h ** 2)

In [5]:
L = np.zeros((2 * m2, 2 * m2))
L[0: m2, 0: m2] = np.identity(m2)
L[0: m2, m2: 2 * m2] = - lambda_ * D
L[m2: 2 * m2, 0: m2] = lambda_ * D
L[m2: 2 * m2, m2: 2 * m2] = np.identity(m2)

In [6]:
x = np.linspace(0, 1, n + 1)
y = np.linspace(0, 1, n + 1)
X, Y = np.meshgrid(x, y)

In [7]:
u1_exact = X * (1 - X) * np.sin(np.pi * Y)
u2_exact = np.sin(np.pi * X) * Y * (1 - Y)
laplace_u1 = -2 * np.sin(np.pi * Y) - np.pi ** 2 * u1_exact
laplace_u2 = -2 * np.sin(np.pi * X) - np.pi ** 2 * u2_exact
f1 = u1_exact - lambda_ * laplace_u2
f2 = lambda_ * laplace_u1 + u2_exact

In [8]:
F = np.concatenate([f1[1:-1, 1:-1].flatten(), f2[1:-1, 1:-1].flatten()])

In [9]:
from scipy import sparse
from scipy.sparse.linalg import spsolve
L_sparse = sparse.csr_matrix(L)
u_numerical = spsolve(L_sparse, F)

In [10]:
u1_numerical = np.zeros_like(u1_exact)
u2_numerical = np.zeros_like(u2_exact)
u1_numerical[1:-1, 1:-1] = u_numerical[0: m2].reshape((m, m))
u2_numerical[1:-1, 1:-1] = u_numerical[m2: 2 * m2].reshape((m, m))

In [11]:
def computeErrors(u_exact, u_pre, printOrNot):
    error = u_exact - u_pre
    l2_norm_abs = np.linalg.norm(error, ord=2) / np.sqrt(error.size)
    max_norm_abs = np.max(np.abs(error))
    l2_norm_rel = np.linalg.norm(error, ord=2) / np.linalg.norm(u_exact, ord=2)
    max_norm_rel = np.max(np.abs(error)) / np.max(np.abs(u_exact))  
    
    l2_norm_rel_percent = l2_norm_rel * 100
    max_norm_rel_percent = max_norm_rel * 100
    
    if printOrNot == True:
        print(f"Absolute L2 Norm Error: {l2_norm_abs:.8f}")
        print(f"Absolute Max Norm Error: {max_norm_abs:.8f}")
        print(f"Relative L2 Norm Error: {l2_norm_rel_percent:.6f}%")
        print(f"Relative Max Norm Error: {max_norm_rel_percent:.6f}%")

In [12]:
computeErrors(u1_exact, u1_numerical, True)

Absolute L2 Norm Error: 0.00003214
Absolute Max Norm Error: 0.00006535
Relative L2 Norm Error: 0.025516%
Relative Max Norm Error: 0.026139%


In [13]:
computeErrors(u2_exact, u2_numerical, True)

Absolute L2 Norm Error: 0.00000032
Absolute Max Norm Error: 0.00000079
Relative L2 Norm Error: 0.000255%
Relative Max Norm Error: 0.000318%
